# Purity State Preparation for Qubits
Quantum Neural Network (QNN) that prepare a dataset of density matrices with uniform purity

In [26]:
import os
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from IPython.display import clear_output

from pyquil import Program
from pyquil.gates import *
from pyquil.api import WavefunctionSimulator

import qutip

from scipy.optimize import minimize

os.chdir("..")
from create_purity_dataset import generate_density_matrices

In [ ]:
%load_ext autoreload
%autoreload 2

In [3]:
np.random.seed(42)

## Constants

### Dataset

In [4]:
data_folder = "data_qubits"

n_qubits_rho = 1
n_qubits_psi = n_qubits_rho * 2 # for purification

size_hilbert = 2**n_qubits_rho

n_samples = 400

### Training

In [ ]:
functional_reg = 1

## Creating the dataset

In [100]:
rhos, purities = generate_density_matrices(n_samples, size_hilbert)

Generate density matrices...
(400, 2, 2)


## State Preparation Network

In [101]:
wf_sim = WavefunctionSimulator()

### Network

In [102]:
def sp_network(params):
    circuit = Program()
    circuit += RZ(params[0], 0)
    circuit += RY(params[1], 0)
    circuit += RZ(params[2], 0)
    circuit += RZ(params[3], 1)
    circuit += RY(params[4], 1)
    circuit += RZ(params[5], 1)
    circuit += CNOT(1,0)
    circuit += RZ(params[6], 0)
    circuit += RY(params[7], 1)
    circuit += CNOT(0,1)
    circuit += RY(params[8], 1)
    circuit += CNOT(1,0)
    circuit += RZ(params[9], 0)
    circuit += RY(params[10], 0)
    circuit += RZ(params[11], 0)
    circuit += RZ(params[12], 1)
    circuit += RY(params[13], 1)
    circuit += RZ(params[14], 1)

    return circuit

### Parameters

In [103]:
params = np.random.normal(size=15, scale=0.01)

### Output

In [106]:
def partial_trace(state):
    qstate = state[0] * qutip.states.ket([0,0]) \
             + state[1] * qutip.states.ket([0,1]) \
             + state[2] * qutip.states.ket([1,0]) \
             + state[3] * qutip.states.ket([1,1]) 
    rho_output = np.array(qstate.ptrace(1).data.todense())
    return rho_output

In [122]:
def get_rho(params):
    circuit = sp_network(params)
    state = list(wf_sim.wavefunction(circuit))
    rho_output = partial_trace(state)
    
    return rho_output

### Cost and optimizer

In [107]:
def purity_fct(rho):
    return np.real(np.trace(rho @ rho))

In [108]:
def purity_mse(rho_output, rho_input):
    return np.mean((purity_fct(rho_output) - purity_fct(rho_input))**2)

In [109]:
def trace_distance(rho1, rho2):
    return np.mean(np.real(np.linalg.eigvalsh(rho1 - rho2)**2))

In [123]:
def cost(params, rho_input):
    rho_output = get_rho(params)
    
    return trace_distance(rho_output, rho_input) + functional_reg * purity_mse(rho_output, rho_input)

In [124]:
cost(params, rhos[0])

0.520065817038432

In [121]:
minimize(lambda params: cost(params, rhos[0]), params, method='L-BFGS-B')

      fun: 1.9719467942108008e-10
 hess_inv: <15x15 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 2.58088106e-13, -1.68260539e-07,  2.03084268e-07, -3.29915817e-14,
       -2.82366600e-07, -9.12958975e-06, -9.35946365e-06,  5.37335553e-06,
       -3.79367659e-06, -1.23703310e-07,  1.89681302e-07, -1.23598343e-07,
       -1.29350368e-13, -1.54501829e-13, -2.21216530e-13])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 272
      nit: 8
   status: 0
  success: True
        x: array([-3.02007643e-04, -1.12120444e-01,  4.93856674e-03, -2.37786788e-02,
        2.34361988e+00,  2.60772515e-02,  7.46004651e-03, -1.45827505e+00,
        2.04234497e-01,  1.66953114e-02,  2.65574575e+00, -6.16405169e-02,
        7.04911607e-03,  2.66710433e-03, -2.07188766e-02])

In [118]:
partial_trace(list(wf_sim.wavefunction(sp_network(result['x']))))

array([[0.47929859+0.j        , 0.01032374-0.01118915j],
       [0.01032374+0.01118915j, 0.52070141+0.j        ]])

In [120]:
rhos[0]

array([[0.47929711+0.j       , 0.0103298 -0.0112017j],
       [0.0103298 +0.0112017j, 0.52070289+0.j       ]])

### Training

In [ ]:
cost_train_list = []
cost_test_list = []
i = 0

In [128]:
params_list = []
cost_list = []
functional_list = []
n_iters = n_iters_max

for i, rho in enumerate(rhos):
    print("\n\n~~~~~~~~~~~~~~~~~~~~~~ Density Matrix {}/{} ~~~~~~~~~~~~~~~~~~~~~~\n".format(i+1, len(rhos)))
    print("Functional: {:.7f}\n\n".format(purity_fct(rho)))

    result = minimize(lambda params: cost(params, rho), params, method='L-BFGS-B')
    params_predict = result['x']
    curr_cost = result['fun']
    curr_functional = purity_fct(get_rho(params_predict))
    
    params_list.append(params_predict)
    cost_list.append(curr_cost)
    functional_list.append(curr_functional)
    
    print('\nCost: {: .7f} −− Functional: {:.7f}'.format(cost_list[-1], functional_list[-1]))
    
    params = params_predict # we change the initialization for the next step



~~~~~~~~~~~~~~~~~~~~~~ Density Matrix 1/400 ~~~~~~~~~~~~~~~~~~~~~~

Functional: 0.5013216



Cost:  0.0000000 −− Functional: 0.5013207


~~~~~~~~~~~~~~~~~~~~~~ Density Matrix 2/400 ~~~~~~~~~~~~~~~~~~~~~~

Functional: 0.5050119



Cost:  0.0000000 −− Functional: 0.5050110


~~~~~~~~~~~~~~~~~~~~~~ Density Matrix 3/400 ~~~~~~~~~~~~~~~~~~~~~~

Functional: 0.5003756



Cost:  0.0000000 −− Functional: 0.5003755


~~~~~~~~~~~~~~~~~~~~~~ Density Matrix 4/400 ~~~~~~~~~~~~~~~~~~~~~~

Functional: 0.5000793



Cost:  0.0000000 −− Functional: 0.5000792


~~~~~~~~~~~~~~~~~~~~~~ Density Matrix 5/400 ~~~~~~~~~~~~~~~~~~~~~~

Functional: 0.5062566



Cost:  0.0000000 −− Functional: 0.5062568


~~~~~~~~~~~~~~~~~~~~~~ Density Matrix 6/400 ~~~~~~~~~~~~~~~~~~~~~~

Functional: 0.5026189



Cost:  0.0000000 −− Functional: 0.5026189


~~~~~~~~~~~~~~~~~~~~~~ Density Matrix 7/400 ~~~~~~~~~~~~~~~~~~~~~~

Functional: 0.5027923



Cost:  0.0000000 −− Functional: 0.5027916


~~~~~~~~~~~~~~~~~~~~~~ Density Matrix 8


Cost:  0.0000000 −− Functional: 0.5784117


~~~~~~~~~~~~~~~~~~~~~~ Density Matrix 61/400 ~~~~~~~~~~~~~~~~~~~~~~

Functional: 0.5790505



Cost:  0.0000000 −− Functional: 0.5790528


~~~~~~~~~~~~~~~~~~~~~~ Density Matrix 62/400 ~~~~~~~~~~~~~~~~~~~~~~

Functional: 0.5730195



Cost:  0.0000000 −− Functional: 0.5730192


~~~~~~~~~~~~~~~~~~~~~~ Density Matrix 63/400 ~~~~~~~~~~~~~~~~~~~~~~

Functional: 0.5759627



Cost:  0.0000000 −− Functional: 0.5759639


~~~~~~~~~~~~~~~~~~~~~~ Density Matrix 64/400 ~~~~~~~~~~~~~~~~~~~~~~

Functional: 0.5775783



Cost:  0.0000000 −− Functional: 0.5775810


~~~~~~~~~~~~~~~~~~~~~~ Density Matrix 65/400 ~~~~~~~~~~~~~~~~~~~~~~

Functional: 0.5855850



Cost:  0.0000000 −− Functional: 0.5855853


~~~~~~~~~~~~~~~~~~~~~~ Density Matrix 66/400 ~~~~~~~~~~~~~~~~~~~~~~

Functional: 0.5810209



Cost:  0.0000000 −− Functional: 0.5810207


~~~~~~~~~~~~~~~~~~~~~~ Density Matrix 67/400 ~~~~~~~~~~~~~~~~~~~~~~

Functional: 0.5852190



Cost:  0.0000000 −− Functional: 0

KeyboardInterrupt: 